# Cortex Analyst Setup for Call Center Analytics

This notebook sets up Cortex Analyst and Cortex Search services to enable intelligent natural language queries over our call center data. We'll create:

- **Semantic Model** for Cortex Analyst (text-to-SQL)
- **Search Service** for Cortex Search (RAG capabilities)
- **Test queries** to validate setup

## Prerequisites
- AI_TRANSCRIBE notebook completed with `comprehensive_call_analysis` table created
- Cortex Analyst and Cortex Search enabled on your account


In [ ]:
# Import required packages
import pandas as pd
from snowflake.snowpark.context import get_active_session
import yaml
import json

# Get active session
session = get_active_session()
session.use_role("call_center_analytics_role")

print(f"❄️ Setting up Cortex Analyst for Call Center Analytics")
print(f"Role: {session.get_current_role()}")
print(f"Database.Schema: {session.get_fully_qualified_current_schema()}")


## Step 1: Create Stage for Models and Setup


In [ ]:
-- Create stage for storing semantic models and configurations
CREATE STAGE IF NOT EXISTS cortex_models
DIRECTORY = (ENABLE = TRUE)
COMMENT = 'Stage for Cortex Analyst semantic models and Cortex Search configurations';


## Step 2: Create Semantic Model for Cortex Analyst


In [ ]:
select * from comprehensive_call_analysis;

In [ ]:
# Create semantic model YAML for Cortex Analyst
semantic_model = {
    'name': 'call_center_analytics_model',
    'description': 'Semantic model for call center analytics with AI transcription data, enabling natural language queries about call performance, agent effectiveness, and customer satisfaction',
    'tables': [{
        'name': 'comprehensive_call_analysis',
        'description': 'Main table containing transcribed call data with AI-extracted insights including sentiment analysis, agent performance metrics, and customer satisfaction scores',
        'base_table': {
            'database': session.get_current_database(),
            'schema': session.get_current_schema(),
            'table': 'comprehensive_call_analysis'
        },
        'dimensions': [
            {
                'name': 'call_id',
                'data_type': 'TEXT',
                'description': 'Unique identifier for each call',
                'expr': 'call_id',
                'synonyms': ['call identifier', 'call number', 'call reference']
            },
            {
                'name': 'agent_name', 
                'data_type': 'TEXT',
                'description': 'Name of the customer service agent handling the call',
                'expr': 'agent_name',
                'synonyms': ['agent', 'representative', 'rep', 'customer service agent', 'support agent']
            },
            {
                'name': 'customer_name',
                'data_type': 'TEXT', 
                'description': 'Name of the customer making the call',
                'expr': 'customer_name',
                'synonyms': ['customer', 'caller', 'client']
            },
            {
                'name': 'primary_intent',
                'data_type': 'TEXT',
                'description': 'Primary reason for the call (billing, technical_support, complaint, information, sales, cancellation, other)',
                'expr': 'primary_intent',
                'synonyms': ['call reason', 'intent', 'purpose', 'call type', 'issue type'],
                'sample_values': ['billing', 'technical_support', 'complaint', 'information', 'sales', 'cancellation', 'other']
            },
            {
                'name': 'customer_satisfaction',
                'data_type': 'TEXT',
                'description': 'Customer satisfaction level determined from call analysis',
                'expr': 'customer_satisfaction',
                'synonyms': ['satisfaction', 'csat', 'customer satisfaction level', 'satisfaction level'],
                'sample_values': ['satisfied', 'neutral', 'dissatisfied']
            },
            {
                'name': 'sentiment_category',
                'data_type': 'TEXT',
                'description': 'Overall sentiment classification of the call conversation',
                'expr': 'sentiment_category',
                'synonyms': ['sentiment', 'mood', 'emotional tone', 'call sentiment'],
                'sample_values': ['POSITIVE', 'NEGATIVE', 'NEUTRAL']
            },
            {
                'name': 'issue_resolved',
                'data_type': 'TEXT',
                'description': 'Whether the customer issue was resolved during the call',
                'expr': 'issue_resolved',
                'synonyms': ['resolved', 'resolution status', 'issue status'],
                'sample_values': ['yes', 'no']
            },
            {
                'name': 'escalation_required',
                'data_type': 'TEXT',
                'description': 'Whether the call required escalation to higher level support',
                'expr': 'escalation_required',
                'synonyms': ['escalated', 'escalation', 'escalation status'],
                'sample_values': ['yes', 'no']
            }
        ],
        'time_dimensions': [
            {
                'name': 'analysis_timestamp',
                'data_type': 'TIMESTAMP_NTZ',
                'description': 'Timestamp when the call analysis was completed',
                'expr': 'analysis_timestamp',
                'synonyms': ['call date', 'call time', 'timestamp', 'analysis date']
            }
        ],
        'facts': [
            {
                'name': 'sentiment_score',
                'data_type': 'NUMBER',
                'description': 'Numerical sentiment score for the call ranging from -1 (very negative) to 1 (very positive)',
                'expr': 'sentiment_score',
                'synonyms': ['sentiment rating', 'emotional score']
            },
            {
                'name': 'agent_performance_score',
                'data_type': 'NUMBER',
                'description': 'Performance score for the agent on a 1-10 scale based on call handling effectiveness',
                'expr': 'agent_performance_score',
                'synonyms': ['agent score', 'performance rating', 'agent rating']
            },
            {
                'name': 'word_count',
                'data_type': 'NUMBER',
                'description': 'Total number of words in the call transcript',
                'expr': 'word_count',
                'synonyms': ['transcript length', 'call length', 'conversation length']
            }
        ],
        'metrics': [
            {
                'name': 'total_calls',
                'description': 'Total number of calls',
                'expr': 'COUNT(*)',
                'synonyms': ['call count', 'number of calls', 'call volume']
            },
            {
                'name': 'avg_sentiment_score',
                'description': 'Average sentiment score across calls',
                'expr': 'AVG(sentiment_score)',
                'synonyms': ['average sentiment', 'mean sentiment', 'sentiment average']
            },
            {
                'name': 'avg_agent_performance',
                'description': 'Average agent performance score',
                'expr': 'AVG(agent_performance_score)',
                'synonyms': ['average agent score', 'mean agent performance', 'agent performance average']
            },
            {
                'name': 'resolution_rate',
                'description': 'Percentage of calls with issues resolved successfully',
                'expr': 'SUM(CASE WHEN issue_resolved = \'yes\' THEN 1 ELSE 0 END) / COUNT(*) * 100',
                'synonyms': ['resolution percentage', 'success rate', 'issue resolution rate']
            },
            {
                'name': 'satisfaction_rate',
                'description': 'Percentage of customers who were satisfied with the call',
                'expr': 'SUM(CASE WHEN customer_satisfaction = \'satisfied\' THEN 1 ELSE 0 END) / COUNT(*) * 100',
                'synonyms': ['satisfaction percentage', 'customer satisfaction rate', 'csat rate']
            },
            {
                'name': 'escalation_rate',
                'description': 'Percentage of calls that required escalation',
                'expr': 'SUM(CASE WHEN escalation_required = \'yes\' THEN 1 ELSE 0 END) / COUNT(*) * 100',
                'synonyms': ['escalation percentage', 'escalation ratio']
            },
            {
                'name': 'positive_sentiment_rate',
                'description': 'Percentage of calls with positive sentiment',
                'expr': 'SUM(CASE WHEN sentiment_category = \'POSITIVE\' THEN 1 ELSE 0 END) / COUNT(*) * 100',
                'synonyms': ['positive sentiment percentage', 'positive call rate']
            }
        ],
        'filters': [
            {
                'name': 'recent_calls',
                'description': 'Filter for calls from the last 30 days',
                'expr': 'analysis_timestamp >= DATEADD(day, -30, CURRENT_DATE())',
                'synonyms': ['recent', 'last 30 days', 'past month']
            },
            {
                'name': 'resolved_calls',
                'description': 'Filter for calls where issues were resolved',
                'expr': 'issue_resolved = \'yes\'',
                'synonyms': ['successful calls', 'resolved issues']
            },
            {
                'name': 'escalated_calls',
                'description': 'Filter for calls that required escalation',
                'expr': 'escalation_required = \'yes\'',
                'synonyms': ['escalated', 'escalation required']
            }
        ]
    }],
    'verified_queries': [
        {
            'name': 'agent_performance_summary',
            'question': 'What is the average performance score by agent?',
            'sql': 'SELECT agent_name, AVG(agent_performance_score) as avg_performance FROM comprehensive_call_analysis GROUP BY agent_name ORDER BY avg_performance DESC'
        },
        {
            'name': 'resolution_rate_by_intent', 
            'question': 'What is the resolution rate by call intent?',
            'sql': 'SELECT primary_intent, SUM(CASE WHEN issue_resolved = \'yes\' THEN 1 ELSE 0 END) / COUNT(*) * 100 as resolution_rate FROM comprehensive_call_analysis GROUP BY primary_intent ORDER BY resolution_rate DESC'
        },
        {
            'name': 'daily_call_volume',
            'question': 'How many calls do we get per day?',
            'sql': 'SELECT DATE(analysis_timestamp) as call_date, COUNT(*) as daily_calls FROM comprehensive_call_analysis GROUP BY DATE(analysis_timestamp) ORDER BY call_date DESC'
        }
    ]
}

# Convert to YAML
semantic_model_yaml = yaml.dump(semantic_model, default_flow_style=False, sort_keys=False)

print("✅ Semantic model created - REMOVED invalid data_type from metrics")
print("Preview of semantic model YAML:")
print(semantic_model_yaml[:500] + "...") 

In [ ]:
# Save semantic model to stage
with open('/tmp/call_center_semantic_model.yaml', 'w') as f:
    f.write(semantic_model_yaml)

# Upload to Snowflake stage
session.file.put('/tmp/call_center_semantic_model.yaml', '@cortex_models/semantic_models/', auto_compress=False, overwrite=True)

print("✅ Semantic model uploaded to stage: @cortex_models/semantic_models/call_center_semantic_model.yaml")


## Step 3: Create Cortex Search Service


In [ ]:
-- Create Cortex Search Service for call transcripts
-- Note: Using current warehouse context, change WAREHOUSE if needed
CREATE OR REPLACE CORTEX SEARCH SERVICE call_center_search
ON transcript_text
ATTRIBUTES call_id, agent_name, customer_name, primary_intent, sentiment_category, customer_satisfaction
WAREHOUSE = GOOGLE_ANALYTICS_DS_WH  -- Change this to your warehouse name if different
TARGET_LAG = '1 minute'
AS (
    SELECT 
        call_id,
        agent_name,
        customer_name, 
        primary_intent,
        sentiment_category,
        customer_satisfaction,
        transcript_text
    FROM comprehensive_call_analysis
    WHERE transcript_text IS NOT NULL
    AND LENGTH(transcript_text) > 50
);


In [ ]:
-- Check the status of the search service
SHOW CORTEX SEARCH SERVICES;


## Step 4: Test Cortex Search Service


In [ ]:
-- Test the search service with a sample query
SELECT
    SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
        'call_center_search',
        '{
            "query": "billing complaint negative sentiment",
            "limit": 5
        }'
    ) as search_results;

## Step 5: Create Additional Views for Analytics


In [ ]:
-- Create a summary view for KPIs
CREATE OR REPLACE VIEW call_center_kpis AS
SELECT 
    COUNT(*) as total_calls,
    COUNT(DISTINCT agent_name) as unique_agents,
    ROUND(AVG(sentiment_score), 3) as avg_sentiment_score,
    ROUND(AVG(agent_performance_score), 1) as avg_agent_performance,
    ROUND(SUM(CASE WHEN issue_resolved = 'yes' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as resolution_rate_pct,
    ROUND(SUM(CASE WHEN customer_satisfaction = 'satisfied' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as satisfaction_rate_pct,
    ROUND(SUM(CASE WHEN escalation_required = 'yes' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as escalation_rate_pct,
    ROUND(SUM(CASE WHEN sentiment_score > 0.1 THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as positive_sentiment_pct,
    MODE(primary_intent) as top_call_intent,
    ROUND(AVG(word_count), 0) as avg_call_length_words
FROM comprehensive_call_analysis
WHERE agent_name IS NOT NULL 
AND agent_name != 'Not Available';


### Additional Views for Streamlit App

The Streamlit application requires additional views for agent performance analysis.


In [ ]:
-- Test our views
SELECT * FROM call_center_kpis;


In [ ]:
-- Create agent performance summary view for Streamlit app
CREATE OR REPLACE VIEW agent_performance_summary AS
SELECT 
    agent_name,
    COUNT(*) as total_calls,
    ROUND(AVG(sentiment_score), 3) as avg_sentiment,
    ROUND(AVG(agent_performance_score), 1) as avg_performance_score,
    
    -- Resolution effectiveness
    SUM(CASE WHEN issue_resolved = 'yes' THEN 1 ELSE 0 END) as resolved_calls,
    ROUND(SUM(CASE WHEN issue_resolved = 'yes' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as resolution_rate,
    
    -- Customer satisfaction
    SUM(CASE WHEN customer_satisfaction = 'satisfied' THEN 1 ELSE 0 END) as satisfied_customers,
    ROUND(SUM(CASE WHEN customer_satisfaction = 'satisfied' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as satisfaction_rate,
    
    -- Escalation patterns
    SUM(CASE WHEN escalation_required = 'yes' THEN 1 ELSE 0 END) as escalations,
    ROUND(SUM(CASE WHEN escalation_required = 'yes' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as escalation_rate
    
FROM comprehensive_call_analysis
WHERE agent_name != 'Not Available' AND agent_name IS NOT NULL
GROUP BY agent_name
ORDER BY avg_performance_score DESC;


## Step 6: Configuration Summary

✅ **Setup Complete!** The following services are now ready:

### 🔍 **Cortex Search Service**
- **Name**: `call_center_search`
- **Purpose**: Semantic search over call transcripts
- **Usage**: Find relevant calls based on natural language queries

### 📊 **Cortex Analyst Semantic Model**
- **Location**: `@cortex_models/semantic_models/call_center_semantic_model.yaml`
- **Purpose**: Enable natural language to SQL queries
- **Usage**: Ask questions about call center metrics in plain English

### 📈 **Views Created**
- `call_center_kpis`: Overall KPI summary
- `agent_performance_summary`: Agent performance metrics for detailed analysis

### 🎯 **Ready for Streamlit App**
The new Streamlit application can now use:
- Cortex Agents with both search and analyst capabilities
- Pre-built views for dashboard KPIs and agent performance
- Rich analytics from AI_TRANSCRIBE data
